# Rutas y modulos

In [18]:
# Importar librerías necesarias
import os  # Para trabajar con rutas del sistema
import pandas as pd  # Para trabajar con dataframes y archivos Excel
import numpy as np  # Para operaciones numéricas
import re  # Para validación de correos electrónicos
from datetime import datetime  # Para trabajar con fechas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from datetime import datetime
import sys  # Acceso a variables y funciones del sistema


# Añadir la carpeta raíz del proyecto al sys.path para importar módulos personalizados
#sys.path.append(os.path.abspath("c:/Users/osmarrincon/Documents/capresoca-data-automation"))
sys.path.append(os.path.abspath(r"C:\Users\crist\Documents\Proyectos Python\capresoca-data-automation"))  # Ruta alternativa (comentada)

# Importar función y clase personalizada del proyecto
from src.file_loader import cargar_maestros_ADRES  # Función para cargar archivos maestros ADRES
from src.data_cleaning import BduaReportProcessor      # Clase para limpiar y normalizar población Maestro ADRES
from src.data_cleaning import DataCleaner # Clase para limpiar y normalizar DataFrames de Pandas

In [19]:
Fecha = "11-11-2025"

In [20]:
# Oficce
# hoja = "Sheet1"
# R_MS_SIe = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\SIE\Aseguramiento\ms_sie\Reporte_Validación Archivos Maestro_2025_09_11.csv"
# R_Pila_movilidad = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\09_Septiempre\09\Dataframe Pila 09-09-2025.xlsx"
# R_Maestro__EPSC25 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0009092025.TXT"
# R_Maestro__EPS025 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2025-02\EPS025MS0009092025.TXT"
# R_Cod_DANE = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Proyecto INOVA\Colab_Notebooks\dashboard\Base de datos\Codigo DANE\Departamentos.txt"


#S_Excel = fr"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\informes\2025\CTO135.2025 Informe  #9\ACTIVIDAD 14\Bases de datos notificaciones telefonicas\MOVILIDAD NO EFECTIVA\Movilidad No Efectiva {Fecha}.xlsx"

In [21]:
# Home
hoja = "Sheet1"
R_MS_SIe = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\SIE\Aseguramiento\ms_sie\Reporte_Validación Archivos Maestro_2025_11_05.csv"
R_Pila_movilidad = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\11_Noviembre\11\Dataframe Pila 11-11-2025.xlsx"
R_Maestro__EPSC25 = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0010112025.TXT"
R_Maestro__EPS025 = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2025-02\EPS025MS0010112025.TXT"
R_Cod_DANE = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Constantes\Departamentos.txt"


S_Excel = fr"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\informes\2025\CTO135.2025 Informe  #11\ACTIVIDAD 14\Bases de datos notificaciones telefonicas\MOVILIDAD NO EFECTIVA\2ER PROCESO\Movilidad No Efectiva {Fecha}.xlsx"

# Cargue Dataframe

In [22]:
# Cargar y combinar los maestros
maestro_ADRES = cargar_maestros_ADRES(R_Maestro__EPS025, R_Maestro__EPSC25)

# Duplicar la columna "MARCASISBENIV+MARCASISBENIII_2" y nombrarla "MARCASISBENIV+MARCASISBENIII"
maestro_ADRES["MARCASISBENIV+MARCASISBENIII_2"] = \
    maestro_ADRES["MARCASISBENIV+MARCASISBENIII"]


# 1. Instanciar el procesador: Se crea un objeto pasando el DataFrame.
#    La jerarquía de población ya está definida por defecto dentro de la clase.
processor = BduaReportProcessor(df=maestro_ADRES)

# 2. Ejecutar la limpieza y asignarla de vuelta.
#    El método retorna un DataFrame completamente nuevo con la columna actualizada.
maestro_ADRES = processor.prioritize_population_markers(
    col_name="MARCASISBENIV+MARCASISBENIII"
)

# ¡Listo! 'maestro_ADRES' ahora contiene los datos limpios.

In [23]:
# Cargar el archivo Excel R_Pila_movilidad en un DataFrame
df_pila_movilidad = pd.read_excel(R_Pila_movilidad, sheet_name=hoja, dtype=str)
df_ms_SIE = pd.read_csv(R_MS_SIe, sep=';', encoding='ansi', dtype=str)
df_cod_dane = pd.read_csv(R_Cod_DANE, sep=';', encoding='UTF-8', dtype=str)

# Normalizar datos

In [24]:
print(df_pila_movilidad['Movilidad'].unique())

['Al dia' 'Relación laboral Reciente' 'Mora' 'S1' 'No sisben' 'Sisben D'
 'Activo Regimen especial' 'UGPP' 'S1-Mora' 'Pencionado' 'S1-UGPP']


In [25]:
# Filtrar registros donde 'Movilidad' contenga 'sin sisben' o 'sisben D' (ignorando mayúsculas/minúsculas)
filtro = df_pila_movilidad['Movilidad'].str.contains('sin sisben', case=False, na=False) | \
         df_pila_movilidad['Movilidad'].str.contains('no sisben', case=False, na=False) | \
         df_pila_movilidad['Movilidad'].str.contains('sisben D', case=False, na=False)

# Mostrar cantidad antes del filtrado
print(f"Registros antes del filtrado: {len(df_pila_movilidad)}")

# Aplicar el filtro
df_pila_movilidad = df_pila_movilidad[filtro]

# Mostrar cantidad después del filtrado
print(f"Registros después del filtrado: {len(df_pila_movilidad)}")

Registros antes del filtrado: 18374
Registros después del filtrado: 1352


In [26]:
df_pila_movilidad = df_pila_movilidad[["Tipo Documento Cotizante", "N° Identificación Cotizante", "Población", "Movilidad"]]

In [27]:
# Asegurar que los identificadores sean string para evitar problemas de merge
maestro_ADRES['TPS_IDN_ID'] = maestro_ADRES['TPS_IDN_ID'].astype(str)
maestro_ADRES['HST_IDN_NUMERO_IDENTIFICACION'] = maestro_ADRES['HST_IDN_NUMERO_IDENTIFICACION'].astype(str)
df_pila_movilidad['Tipo Documento Cotizante'] = df_pila_movilidad['Tipo Documento Cotizante'].astype(str)
df_pila_movilidad['N° Identificación Cotizante'] = df_pila_movilidad['N° Identificación Cotizante'].astype(str)

# Seleccionar columnas a traer del maestro
cols_maestro = [
    "TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION",
    "AFL_PRIMER_APELLIDO", "AFL_SEGUNDO_APELLIDO",
    "AFL_PRIMER_NOMBRE", "AFL_SEGUNDO_NOMBRE",
    "AFL_FECHA_NACIMIENTO", "TPS_GNR_ID", "DPR_ID", "MNC_ID"
]

# Hacer el merge
df_pila_movilidad = df_pila_movilidad.merge(
    maestro_ADRES[cols_maestro],
    left_on=["Tipo Documento Cotizante", "N° Identificación Cotizante"],
    right_on=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"],
    how="left"
)

In [28]:
df_pila_movilidad = df_pila_movilidad.drop(columns=["Tipo Documento Cotizante", "N° Identificación Cotizante"])

In [29]:
# Seleccionar columnas a unir desde df_ms_SIE
cols_contacto = [
    "tipo_documento", "numero_identificacion",
    "celular", "telefono_1", "telefono_2", "correo_electronico"
]

# Realizar el merge usando los identificadores correspondientes
df_pila_movilidad = df_pila_movilidad.merge(
    df_ms_SIE[cols_contacto],
    left_on=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"],
    right_on=["tipo_documento", "numero_identificacion"],
    how="left"
)

# Eliminar las columnas de identificadores provenientes de df_ms_SIE
df_pila_movilidad = df_pila_movilidad.drop(columns=["tipo_documento", "numero_identificacion"])

In [30]:
def limpiar_telefono(numero):
    """
    Limpia y valida un número de teléfono colombiano.
    - Elimina caracteres no numéricos.
    - Debe tener 10 dígitos y empezar por '3'.
    - Si no cumple, retorna ''.
    """
    if pd.isnull(numero):
        return ''
    # Eliminar espacios, signos y dejar solo números
    solo_numeros = re.sub(r'\D', '', str(numero))
    # Validar longitud y prefijo
    if len(solo_numeros) == 10 and solo_numeros.startswith('3'):
        return solo_numeros
    return ''

# Aplicar la función a las columnas de teléfono
for col in ["celular", "telefono_1", "telefono_2"]:
    df_pila_movilidad[col] = df_pila_movilidad[col].apply(limpiar_telefono)

In [31]:
def limpiar_correo(correo):
    """
    Limpia y valida un correo electrónico:
    - Elimina espacios y convierte a minúsculas.
    - Descarta correos con palabras clave como 'actualizar', 'notiene', 'sincorreo', etc.
    - Valida la estructura básica de correo electrónico.
    - Si no es válido, retorna ''.
    """
    if pd.isnull(correo):
        return ''
    correo = str(correo).strip().lower()
    # Palabras clave no válidas
    palabras_invalidas = [
        'actualizar', 'notiene', 'no tiene', 'sincorreo', 'sin correo', 'noaplica', 'no aplica', 'ninguno', 'noexiste', 'no existe'
    ]
    for palabra in palabras_invalidas:
        if palabra in correo:
            return ''
    # Validar estructura básica de correo
    patron = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    if re.match(patron, correo):
        return correo
    return ''

# Aplicar la función a la columna 'correo_electronico'
df_pila_movilidad['correo_electronico'] = df_pila_movilidad['correo_electronico'].apply(limpiar_correo)

In [32]:
# Realizar el merge para agregar los nombres de departamento y municipio
df_pila_movilidad = df_pila_movilidad.merge(
    df_cod_dane[['Código Deapartamento', 'Código Municipio', 'Nombre Departamento', 'Nombre Municipio']],
    left_on=['DPR_ID', 'MNC_ID'],
    right_on=['Código Deapartamento', 'Código Municipio'],
    how='left'
)

# Eliminar las columnas de id provenientes de df_cod_dane
df_pila_movilidad = df_pila_movilidad.drop(columns=['Código Deapartamento', 'Código Municipio'])

# Guardar dataframes

In [33]:
df_pila_movilidad.columns

Index(['Población', 'Movilidad', 'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION',
       'AFL_PRIMER_APELLIDO', 'AFL_SEGUNDO_APELLIDO', 'AFL_PRIMER_NOMBRE',
       'AFL_SEGUNDO_NOMBRE', 'AFL_FECHA_NACIMIENTO', 'TPS_GNR_ID', 'DPR_ID',
       'MNC_ID', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico',
       'Nombre Departamento', 'Nombre Municipio'],
      dtype='object')

In [34]:
with pd.ExcelWriter(S_Excel, engine='openpyxl') as writer:
    for movilidad, grupo in df_pila_movilidad.groupby('Movilidad'):
        # Limpiar el nombre de la hoja para evitar caracteres no permitidos
        nombre_hoja = str(movilidad)[:31].replace('/', '_').replace('\\', '_').replace('*', '_').replace('?', '_').replace('[', '_').replace(']', '_')
        grupo.to_excel(writer, sheet_name=nombre_hoja, index=False)